In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from dotenv import load_dotenv
from random import shuffle
import pathlib
import json
import shutil

import numpy as np
from sklearn.cluster import MiniBatchKMeans
import traceback
import faiss
import torch, platform

now_dir = os.getcwd()
sys.path.append(now_dir)
load_dotenv()

True

In [3]:
from time import sleep
from subprocess import Popen
# from random import shuffle
# import warnings
# import traceback
import threading
# import shutil
import logging

In [4]:
# config = Config()

logger = logging.getLogger(__name__)
sr_dict = {
    "32k": 32000,
    "40k": 40000,
    "48k": 48000,
}

def if_done(done, p):
    while 1:
        if p.poll() is None:
            sleep(0.5)
        else:
            break
    done[0] = True

In [5]:
#from infer-web.py
def preprocess_dataset(trainset_dir, exp_dir, sr, n_p):
    sr = sr_dict[sr]
    print('making dirs')
    os.makedirs("%s/logs/%s" % (now_dir, exp_dir), exist_ok=True)
    f = open("%s/logs/%s/preprocess.log" % (now_dir, exp_dir), "w")
    f.close()

    # Taken from Config defaults
    python_cmd = "python"
    preprocess_per = 3.7
    noparallel = False

    cmd = '"%s" infer/modules/train/preprocess.py "%s" %s %s "%s/logs/%s" %s %.1f' % (
        #config.python_cmd,
        python_cmd,
        trainset_dir,
        sr,
        n_p,
        now_dir,
        exp_dir,
        #config.noparallel,
        #config.preprocess_per,
        noparallel,
        preprocess_per,
    )
    python_cmd = "python"
    logger.info("Execute: " + cmd)
    p = Popen(cmd, shell=True)

    # threading stuff 
    # 煞笔gr, popen read都非得全跑完了再一次性读取, 不用gr就正常读一句输出一句;只能额外弄出一个文本流定时读
    
    # done = [False]
    # threading.Thread(
    #     target=if_done,
    #     args=(
    #         done,
    #         p,
    #     ),
    # ).start()
    # while 1:
    #     with open("%s/logs/%s/preprocess.log" % (now_dir, exp_dir), "r") as f:
    #         yield (f.read())
    #     sleep(1)
    #     if done[0]:
    #         break
    # with open("%s/logs/%s/preprocess.log" % (now_dir, exp_dir), "r") as f:
    #     log = f.read()
    # logger.info(log)
    # yield log

In [6]:
def extract_f0_feature(gpus, n_p, f0method, if_f0, exp_dir, version19, gpus_rmvpe):
    gpus = gpus.split("-")
    os.makedirs("%s/logs/%s" % (now_dir, exp_dir), exist_ok=True)
    f = open("%s/logs/%s/extract_f0_feature.log" % (now_dir, exp_dir), "w")
    f.close()

    python_cmd = "python"
    is_half = False
    device = 'mps'

    if if_f0:
        if f0method != "rmvpe_gpu":
            cmd = (
                '"%s" infer/modules/train/extract/extract_f0_print.py "%s/logs/%s" %s %s'
                % (
                    #config.python_cmd,
                    python_cmd,
                    now_dir,
                    exp_dir,
                    n_p,
                    f0method,
                )
            )
            logger.info("Execute: " + cmd)
            p = Popen(
                cmd, shell=True, cwd=now_dir
            )  # , stdin=PIPE, stdout=PIPE,stderr=PIPE
            # 煞笔gr, popen read都非得全跑完了再一次性读取, 不用gr就正常读一句输出一句;只能额外弄出一个文本流定时读
            # done = [False]
            # threading.Thread(
            #     target=if_done,
            #     args=(
            #         done,
            #         p,
            #     ),
            # ).start()

    leng = len(gpus)
    print(f'gpus : {gpus}')
    print(f'leng = {leng}')
    ps = []
    for idx, n_g in enumerate(gpus):
        cmd = (
            '"%s" infer/modules/train/extract_feature_print.py %s %s %s %s "%s/logs/%s" %s %s'
            % (
                #config.python_cmd,
                python_cmd,
                #config.device,
                device,
                leng,
                idx,
                n_g,
                now_dir,
                exp_dir,
                version19,
                #config.is_half,
                is_half
            )
        )
        logger.info("Execute: " + cmd)
        p = Popen(
            cmd, shell=True, cwd=now_dir
        )  # , shell=True, stdin=PIPE, stdout=PIPE, stderr=PIPE, cwd=now_dir
        ps.append(p)

In [7]:
version_config_list = [
    "v1/32k.json",
    "v1/40k.json",
    "v1/48k.json",
    "v2/48k.json",
    "v2/32k.json",
]

def load_config_json() -> dict:
    d = {}
    for config_file in version_config_list:
        p = f"configs/inuse/{config_file}"
        if not os.path.exists(p):
            shutil.copy(f"configs/{config_file}", p)
        with open(f"configs/inuse/{config_file}", "r") as f:
            d[config_file] = json.load(f)
    return d

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):

    json_config = load_config_json()
    python_cmd = 'python'

    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))
    logger.debug("Write filelist done")
    # 生成config#无需生成config
    # cmd = python_cmd + " train_nsf_sim_cache_sid_load_pretrain.py -e mi-test -sr 40k -f0 1 -bs 4 -g 0 -te 10 -se 5 -pg pretrained/f0G40k.pth -pd pretrained/f0D40k.pth -l 1 -c 0"
    logger.info("Use gpus: %s", str(gpus16))
    if pretrained_G14 == "":
        logger.info("No pretrained Generator")
    if pretrained_D15 == "":
        logger.info("No pretrained Discriminator")
    if version19 == "v1" or sr2 == "40k":
        config_path = "v1/%s.json" % sr2
    else:
        config_path = "v2/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            json.dump(
                #config.json_config[config_path],
                json_config[config_path],
                f,
                ensure_ascii=False,
                indent=4,
                sort_keys=True,
            )
            f.write("\n")
    if gpus16:
        cmd = (
            '"%s" infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
            % (
                #config.python_cmd,
                python_cmd, 
                exp_dir1,
                sr2,
                1 if if_f0_3 else 0,
                batch_size12,
                gpus16,
                total_epoch11,
                save_epoch10,
                "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
                "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
                #1 if if_save_latest13 == i18n("是") else 0,
                #1 if if_cache_gpu17 == i18n("是") else 0,
                #1 if if_save_every_weights18 == i18n("是") else 0,
                1 if if_save_latest13 == 'Yes' else 0,
                1 if if_cache_gpu17 == 'Yes' else 0,
                1 if if_save_every_weights18 == 'Yes' else 0,
                version19,
            )
        )
    else:
        cmd = (
            '"%s" infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
            % (
                #config.python_cmd,
                python_cmd,
                exp_dir1,
                sr2,
                1 if if_f0_3 else 0,
                batch_size12,
                total_epoch11,
                save_epoch10,
                "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
                "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
                # 1 if if_save_latest13 == i18n("是") else 0,
                # 1 if if_cache_gpu17 == i18n("是") else 0,
                # 1 if if_save_every_weights18 == i18n("是") else 0,
                1 if if_save_latest13 == 'Yes' else 0,
                1 if if_cache_gpu17 == 'Yes' else 0,
                1 if if_save_every_weights18 == 'Yes' else 0,
                version19,
            )
        )
    logger.info("Execute: " + cmd)
    p = Popen(cmd, shell=True, cwd=now_dir)
    p.wait()
    return "训练结束, 您可查看控制台训练日志或实验文件夹下的train.log"

In [8]:
def train_index(exp_dir1, version19):
    # exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)

    print('here')
    outside_index_root = os.getenv("outside_index_root")
    n_cpu = 8

    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "请先进行特征提取!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "请先进行特征提取！"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        #yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    #batch_size=256 * config.n_cpu,
                    batch_size=256 * n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            #yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    #yield "\n".join(infos)
    
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    # index = faiss.index_factory(256if version19=="v1"else 768, "IVF%s,PQ128x4fs,RFlat"%n_ivf)
    infos.append("training")
    
    #yield "\n".join(infos)
    
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append("adding")
    #yield "\n".join(infos)
    
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "成功构建索引 added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )
    try:
        link = os.link if platform.system() == "Windows" else os.symlink
        link(
            "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
            % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
            "%s/%s_IVF%s_Flat_nprobe_%s_%s_%s.index"
            % (
                outside_index_root,
                exp_dir1,
                n_ivf,
                index_ivf.nprobe,
                exp_dir1,
                version19,
            ),
        )
        infos.append("链接索引到外部-%s" % (outside_index_root))
    except:
        infos.append("链接索引到外部-%s失败" % (outside_index_root))

    print('Training index finished!')
    # faiss.write_index(index, '%s/added_IVF%s_Flat_FastScan_%s.index'%(exp_dir,n_ivf,version19))
    # infos.append("成功构建索引，added_IVF%s_Flat_FastScan_%s.index"%(n_ivf,version19))
    #yield "\n".join(infos)

In [9]:
root = '/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/'
trainset_dir4 = f'{root}/data/small_dataset'
exp_dir1 = 'script-test'
sr2 = "40k"
np7 = 6

#"Select the pitch extraction algorithm: when extracting singing, 
# you can use 'pm' to speed up. For high-quality speech with fast 
# performance, but worse CPU usage, you can use 'dio'. 'harvest' 
# results in better quality but is slower.  'rmvpe' has the best 
# results and consumes less CPU/GPU",
choices_f0method8=["pm", "harvest", "dio", "rmvpe", "rmvpe_gpu"]
f0method8 = "pm"

# "Enter the GPU index(es) separated by '-', e.g., 
# 0-1-2 to use GPU 0, 1, and 2:"
gpus6 = '' 

# "Whether the model has pitch guidance 
# (required for singing, optional for speech):"
if_f0_3 = True

# "Version"
choices_version19 =["v1", "v2"]
version19 = "v2"

# "Enter the GPU index(es) separated by '-', e.g., 
# 0-0-1 to use 2 processes in GPU0 and 1 process in GPU1",
gpus_rmvpe = '-' # for no gpus

# speaker id???
spk_id5 = 0

# Save frequency (5)
save_epoch10 = 5

#Total training epochs (20)
total_epoch11 = 2 #20

# Batch size per GPU (1)
batch_size12 = 1

# Save only the latest '.ckpt' file to save disk space: (No)
if_save_latest13 = 'No'

# Cache all training sets to GPU memory. Caching small datasets 
# (less than 10 minutes) can speed up training, but caching large datasets 
# will consume a lot of GPU memory and may not provide much speed improvement: (No)
if_cache_gpu17 = 'No'

# Save a small final model to the 'weights' folder at each save point: (No)
if_save_every_weights18 = 'No'

# Load pre-trained base model G path: (assets/pretrained_v2/f0G40k.pth)
pretrained_G14 = 'assets/pretrained_v2/f0G40k.pth'

# Load pre-trained base model D path: (assets/pretrained_v2/f0D40k.pth)
pretrained_D15 = 'assets/pretrained_v2/f0D40k.pth'

# Enter the GPU index(es) separated by '-', e.g., 0-1-2 to use GPU 0, 1, 
# and 2: (None but -??)
gpus16 = '-'

In [10]:
preprocess_dataset(trainset_dir4, exp_dir1, sr2, np7)

making dirs


/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI//data/small_dataset 40000 6 /Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/logs/script-test False 3.7
start preprocess
/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI//data/small_dataset 40000 6 /Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/logs/script-test False 3.7
/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI//data/small_dataset 40000 6 /Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/logs/script-test False 3.7
/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI//data/small_dataset 40000 6 /Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/logs/script-test False 3.7
/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI//data/small_dataset 40000 6 /Users/tomasand

In [11]:
extract_f0_feature(gpus6,
                    np7,
                    f0method8,
                    if_f0_3,
                    exp_dir1,
                    version19,
                    gpus_rmvpe)

gpus : ['']
leng = 1


infer/modules/train/extract/extract_f0_print.py /Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/logs/script-test 6 pm
no-f0-todo
no-f0-todo
no-f0-todo
no-f0-todo
no-f0-todo
no-f0-todo
infer/modules/train/extract_feature_print.py mps 1 0 /Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/logs/script-test v2 False
exp_dir: /Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/logs/script-test
load model(s) from assets/hubert/hubert_base.pt


/Users/tomasandrade/miniconda3/envs/rvc/lib/python3.9/site-packages/fairseq/checkpoint_utils.py:315: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_

/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI//data/small_dataset/0_6.wav	-> Success
/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI//data/small_dataset/0_3.wav	-> Success
/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI//data/small_dataset/0_4.wav	-> Success
/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI//data/small_dataset/0_1.wav	-> Success
end preprocess
move model to mps
all-feature-4


/Users/tomasandrade/miniconda3/envs/rvc/lib/python3.9/site-packages/fairseq/utils.py:744: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Indexing.mm:335.)
  tensor[indices] = value


now-4,all-0,0_1.wav,(184, 768)
now-4,all-1,1_1.wav,(184, 768)
now-4,all-2,2_1.wav,(184, 768)
now-4,all-3,3_1.wav,(46, 768)
all-feature-done


In [12]:
click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
)

INFO:script-test:{'data': {'filter_length': 2048, 'hop_length': 400, 'max_wav_value': 32768.0, 'mel_fmax': None, 'mel_fmin': 0.0, 'n_mel_channels': 125, 'sampling_rate': 40000, 'win_length': 2048, 'training_files': './logs/script-test/filelist.txt'}, 'model': {'filter_channels': 768, 'gin_channels': 256, 'hidden_channels': 192, 'inter_channels': 192, 'kernel_size': 3, 'n_heads': 2, 'n_layers': 6, 'p_dropout': 0, 'resblock': '1', 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'resblock_kernel_sizes': [3, 7, 11], 'spk_embed_dim': 109, 'upsample_initial_channel': 512, 'upsample_kernel_sizes': [16, 16, 4, 4], 'upsample_rates': [10, 10, 2, 2], 'use_spectral_norm': False}, 'train': {'batch_size': 1, 'betas': [0.8, 0.99], 'c_kl': 1.0, 'c_mel': 45, 'epochs': 20000, 'eps': 1e-09, 'fp16_run': False, 'init_lr_ratio': 1, 'learning_rate': 0.0001, 'log_interval': 200, 'lr_decay': 0.999875, 'seed': 1234, 'segment_size': 12800, 'warmup_epochs': 0}, 'model_dir': './logs/script-test', 'ex

/Users/tomasandrade/miniconda3/envs/rvc/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


DEBUG:infer.lib.infer_pack.models:gin_channels: 256, self.spk_embed_dim: 109
INFO:script-test:loaded pretrained assets/pretrained_v2/f0G40k.pth
INFO:script-test:<All keys matched successfully>
INFO:script-test:loaded pretrained assets/pretrained_v2/f0D40k.pth
INFO:script-test:<All keys matched successfully>


/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/infer/modules/train/train.py:231: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(hp

INFO:script-test:Train Epoch: 1 [0%]
INFO:script-test:[0, 0.0001]
INFO:script-test:loss_disc=4.444, loss_gen=1.620, loss_fm=1.272,loss_mel=27.028, loss_kl=9.000
DEBUG:matplotlib:matplotlib data path: /Users/tomasandrade/miniconda3/envs/rvc/lib/python3.9/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/Users/tomasandrade/.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is darwin


...

INFO:script-test:====> Epoch: 1 [2024-12-28 19:12:53] | (0:00:05.753897)


/Users/tomasandrade/Documents/BSC/ICHOIR/Retrieval-based-Voice-Conversion-WebUI/infer/lib/train/data_utils.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  spec = torch

INFO:script-test:====> Epoch: 2 [2024-12-28 19:12:56] | (0:00:03.034633)
INFO:script-test:Training is done. The program is closed.
INFO:script-test:saving final ckpt:Success.


'训练结束, 您可查看控制台训练日志或实验文件夹下的train.log'

/Users/tomasandrade/miniconda3/envs/rvc/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 20 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


In [13]:
train_index(exp_dir1, version19)

here
Training index finished!
